### Book Information

- This whole scraping I am using BeautifulSoup as scraping tool
- The website I scraped is 'https://www.waterstones.com/books/search'
- Fristly import necessary libraies
- Run the script to get data
- write_to_pickle function to save the scraped data as pickle file

In [1]:
from bs4 import BeautifulSoup
from numpy import random
from datetime import datetime
import requests
import pandas as pd
import os
import csv


def write_to_pickle(name, data): ### This function takes whatever list/dict into dataframe and convert it into a pickle file
    file_name = f'{name}.pkl'
    
    if file_name in os.listdir():
        return 'File already exist'

    df = pd.DataFrame(data)
    df.to_pickle(file_name)


def get_book_links(page_number=1): ### This function takes page_number as input, to get the all books on each page
    url = f'https://www.waterstones.com/books/search/page/{page_number}'
    r = requests.get(url)
    soup = BeautifulSoup(r.text)

    book_list = []
    items = soup.find_all('div',class_='info-wrap')

    for item in items:
        book = {}

        book['title'] = item.find('div', class_='title-wrap').text.strip()
        book['author'] = item.find('span', class_='author').text.strip()
        book['price'] = item.find('span', class_='price').text[1:]
        book['book_format'] = item.find_all('span', class_='format')[1].text.strip()
        book['book_link'] = 'https://www.waterstones.com' + item.find('a')['href']
        book['page_url'] = url
        book_list.append(book)
    
    assert len(book_list) >0, 'empty book list returned'
    return book_list

### Get basic information and URLs about the books

- First scrape the whole page(24 books on one page)
- The information are [title, author, price, book format, book link, each page url
- Changing the page number for getting more books
- For every 1894 books, write them to pickle file


In [ ]:
print('START:', datetime.now())

books = []

# total 20834 pages (it may be best to break this scrape up)
for i in range(1, 20834 + 1): 
    try:
        books.append(get_book_links(i))

        if i % 1894 == 0:
            books = [item for sublist in books for item in sublist]
            write_to_pickle(f'book_links_{i}', books)
            books = []

    except Exception as err: ### write to error file if there is an error
        with open('errors.txt', 'a') as f:
            f.write(f'{i}:{err}\n')

print('FINISH:', datetime.now())